In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np


from gridworld_env_layout import GridWorldMDPFromLayoutEnv
from agent.q_learning_agent import ValueIteration


In [ ]:
# Main Steps to implement atom based machine teaching
## 